In [3]:
import os
import sys
# sys.path.append('../')

In [4]:
import cv2
import random
import torch
import torch.nn as nn
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from model.retinanet import get_model
from utils import soft_nms

ModuleNotFoundError: No module named 'backbone'

In [ ]:
id2label = {0: '1', 1:'2', 2:'3', 3:'4', 4:'5', 5:'6', 6:'7', 7:'8', 8:'9', 9:'0'}
coco_classes=['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
color_map = [(np.random.rand(), np.random.rand(), np.random.rand()) for _ in range(len(coco_classes))]

In [ ]:
from collections import OrderedDict

model_path = "../checkpoints/coco/epoch_33_loss_0.73568.pth"
retinanet = get_model(num_classes=80, pretrained=False)
state_dict = torch.load(model_path)

new_state_dictBA = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dictBA[name] = v

retinanet.load_state_dict(new_state_dictBA)
retinanet.eval()

In [ ]:
import random
img_root = '/home/workspace/chencheng/Learning/ObjectDetection/Datasets/CoCodataset/train2017/'
img_lists = os.listdir(img_root)
plt.figure(dpi=180)
for i, img_path in enumerate(random.sample(img_lists, 2)):
    img_path = os.path.join(img_root, img_path)
    img = cv2.imread(img_path)[..., ::-1]
    # img = cv2.resize(img, (640, 480))
    # plt.imshow(img)

    mean=(0.485, 0.456, 0.406)
    std=(0.229, 0.224, 0.225)
    normalized_imgs = (img / 255 - mean) / std 

    resize_wh = (1024, 768)
    ori_h, ori_w = normalized_imgs.shape[:2]
    scale = resize_wh[1] / ori_h
    if scale * ori_w > resize_wh[0]:
        scale = resize_wh[0] / ori_w
    new_h, new_w = int(scale * ori_h), int(scale * ori_w)
    img_resized = cv2.resize(normalized_imgs, (new_w, new_h))

    img_paded = np.zeros([resize_wh[1], resize_wh[0], 3])
    img_paded[:new_h, :new_w, :] = img_resized

    img_input = torch.from_numpy(img_paded).permute(2, 0, 1).unsqueeze(dim=0).float()

    features, regression, classification, anchors = retinanet(img_input)

    from model.utils import BBoxTransform, ClipBoxes, nms_with_class, nms_without_class
    regressBoxes = BBoxTransform()
    clipBoxes = ClipBoxes()

    trans_anchors = regressBoxes(anchors, regression)
    transformed_anchors = clipBoxes(trans_anchors, img_input)

    scores = torch.max(classification, dim=2, keepdim=True)[0]  # 计算anchor的最高得分
    scores_over_thresh = (scores > 0.25)[:, :, 0]
    print(scores_over_thresh.sum())

    classification_per = classification[0, scores_over_thresh[0, :], ...].permute(1, 0)  # 将框对应的分类信息取出来

    transformed_anchors_per = transformed_anchors[0, scores_over_thresh[0, :], ...]  # 将框取出来
    scores_per = scores[0, scores_over_thresh[0, :], ...]
    scores_, classes_ = classification_per.max(dim=0)  # 获得每个框的得分以及对应的类别
    # anchors_nms_idx = nms_with_class(transformed_anchors_per, scores_per[:, 0],classes_, iou_threshold=0.3)
    anchors_nms_idx = nms_without_class(transformed_anchors_per, scores_per[:, 0], iou_threshold=0.3)
    # anchors_nms_idx = soft_nms(transformed_anchors_per, scores_per[:, 0], thresh=0.25)
    
    after_nms_anchor = transformed_anchors_per[anchors_nms_idx]
    after_num_class = classes_[anchors_nms_idx]

    map2ori_bbox = (after_nms_anchor / scale).cpu().detach().numpy()
    filtered_classes = after_num_class.cpu().detach().numpy()

    plt.subplot(1, 2, i+1)
    plt.imshow(img)
    for idx, rect in enumerate(map2ori_bbox):
        x1 = rect[0]
        y1 = rect[1]
        x2 = rect[2]
        y2 = rect[3]
        w = x2 - x1
        h = y2 - y1
        color=color_map[filtered_classes[idx]]
        plt.gca().add_patch(mpl.patches.Rectangle((x1, y1), w, h, edgecolor=color, fill=False))
        plt.text(x1+2, y1-5, coco_classes[filtered_classes[idx]], \
                 fontsize=6,
                 bbox=dict(boxstyle='square,pad=0.2',fc=color, lw=0, alpha=0.5),wrap=True)
plt.show()